In [33]:
import pandas as pd
import numpy as np
import pandapower as pp
import pandapower.networks as pn
import geopandas
import matplotlib.pyplot as plt
import math
from pandapower.plotting.plotly import simple_plotly

In [34]:
net = pn.GBreducednetwork()
pp.runpp(net)

In [35]:
print(net)

This pandapower network includes the following parameter tables:
   - bus (29 elements)
   - load (29 elements)
   - sgen (1 element)
   - gen (64 elements)
   - ext_grid (1 element)
   - line (86 elements)
   - trafo (13 elements)
   - poly_cost (66 elements)
   - bus_geodata (29 elements)
 and the following results tables:
   - res_bus (29 elements)
   - res_line (86 elements)
   - res_trafo (13 elements)
   - res_ext_grid (1 element)
   - res_load (29 elements)
   - res_sgen (1 element)
   - res_gen (64 elements)


In [36]:
net.res_line

,p_from_mw,q_from_mvar,p_to_mw,q_to_mvar,pl_mw,ql_mvar,i_from_ka,i_to_ka,i_ka,vm_from_pu,va_from_degree,vm_to_pu,va_to_degree,loading_percent
0,160.691174,-97.564623,-156.479286,95.909358,4.211888,-1.655265,0.394678,0.385319,0.394678,1.000000,102.162911,1.000000,99.668670,35.807755
1,160.691174,-107.504623,-156.479286,85.969358,4.211888,-21.535265,0.405901,0.374837,0.405901,1.000000,102.162911,1.000000,99.668670,36.825948
10,44.964448,-13.345388,-44.903739,-11.997366,0.060709,-25.342754,0.067699,0.067086,0.067699,1.000000,72.744179,1.000000,67.586992,4.937169
11,44.964448,-13.345388,-44.903739,-11.997366,0.060709,-25.342754,0.067699,0.067086,0.067699,1.000000,72.744179,1.000000,67.586992,4.937169
12,1429.453819,47.195842,-1412.234885,250.010607,17.218934,297.206449,2.076837,2.070081,2.076837,0.993994,72.662113,1.000000,58.125501,46.868881
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
82,2562.131374,82.448386,-2537.150130,354.983825,24.981243,437.432212,3.700032,3.697731,3.700032,1.000000,10.474464,1.000000,0.000000,83.500248
83,2562.131374,82.448386,-2537.150130,354.983825,24.981243,437.432212,3.700032,3.697731,3.700032,1.000000,10.474464,1.000000,0.000000,83.500248
84,672.300534,-41.986613,-669.992211,44.014555,2.308323,2.027942,0.972273,0.969135,0.972273,1.000000,13.549431,1.000000,10.474464,24.230588
85,672.300534,-41.986613,-669.992211,44.014555,2.308323,2.027942,0.972273,0.969135,0.972273,1.000000,13.549431,1.000000,10.474464,24.230588


In [40]:
#p_load_sq = (net.load.p_mw)**2
#q_load_sq = (net.load.q_mvar)**2

#pq_load = (p_load_sq + q_load_sq)**0.5
#pq_load_sum = pq_load.sum()

p_load_sum = net.load.p_mw.sum()

scaling_factor = 1

# p_load_sq_sum = p_load_sq.sum()
# q_load_sq_sum = q_load_sq.sum()
# load_power_sum = (p_load_sq_sum + q_load_sq_sum)

print ("Load in GBreducednetwork =", int(pq_load_sum), "W")
print ("To replicate this system loading and generation, GBreducednetwork is multiplied by the scaling factor of 1")
# because the actual maximum load is around 52,000 and the one used in the package is even higher than this, therefore
# it would only make sense to use the larger load (which is embedded inside the code already)

# Scaling system to represent worst case loads
net.load.scaling = scaling_factor
net.gen.scaling = scaling_factor
net.sgen.scaling = scaling_factor

df1 = pd.DataFrame()
x = range (0,3)#len(net.res_line))
for line in x:
    print (line)
    pp.drop_lines (net, lines = [line])
    
    try:
        pp.runpp(net)
        
        #get the 3 lines with largest loading percent and list them in df1
        loading_percent = (net.res_line.loading_percent)
        loading_percent_max = loading_percent.nlargest(3)
        df1.at [line, "First_loading_percent_max"] = float (loading_percent_max.iloc[0])
        df1.at [line, "First_loading_percent_max_index"] = float (loading_percent_max.index[0])
        df1.at [line, "Second_loading_percent_max"] = float (loading_percent_max.iloc[1])
        df1.at [line, "Second_loading_percent_max_index"] = float (loading_percent_max.index[1])
        df1.at [line, "Third_loading_percent_max"] = float (loading_percent_max.iloc[2])
        df1.at [line, "Third_loading_percent_max_index"] = float (loading_percent_max.index[2])
        
        loading_percent_exceed_100_occurance = sum (i > 100 for i in loading_percent)
        df1.at [line, "Loading_percent_exceed_100_occurance"] = float (loading_percent_exceed_100_occurance)
        
        loading_percent = (net.res_line.loading_percent)
        loading_percent_ave = loading_percent.sum() / len(net.res_lone.loading_percent)
        df1.at [line, "loading_percent_ave"] = float (loading (loading_percent_ave))
        
#   loading_percent = (net.res_line.loading_percent)
#   loading_percent_max = loading_percent.max()
#   df1.at[line,'loading_percent_max'] = float(loading_percent_max)
        
#  loading_percent_max_index = loading_percent.idxmax()
#  df1.at[line,'loading_percent_max_index'] = float(loading_percent_max_index)
        
        line_power = abs(net.res_line.p_from_mw)
        line_power_sum = line_power.sum()
        df1.at [line, "line_power_sum"] = float(line_power_sum)
        
        line_power = abs(net.res_line.p_from_mw)
        line_power_max = line_power.max()
        df1.at [line, "line_power_max"] = float(line_power_max)
        
        volt_magto = (net.res_line.vm_to_pu)
        volt_magto_max = volt_magto.max()
        df1.at[line, "volt_magto_max"] = float(volt_magto_max)
        
        volt_magfrom = (net.res_line.vm_from_pu)
        volt_magfrom_max = volt_magfrom.max()
        df1.at[line, "volt_magfrom_max"] = float(volt_magfrom_max)
        
    except:
        error_variable = float("0")
        df1.at[line, "loading_percent_ave"] = error_variable
        df1.at[line, "loading_percent_max"] = error_variable  
        df1.at[line, "line_power_sum"] = error_variable
        df1.at[line, "line_power_max"] = error_variable
        df1.at[line, "volt_magto_max"] = error_variable
        df1.at[line, "volt_magfrom_max"] = error_variable
         
    net = pn.GBreducednetwork()
    net.load.scaling = scaling_factor
    net.gen.scaling = scaling_factor
    net.sgen.scaling = scaling_factor

df1.to_csv("Results.csv")

#pp.to_excel(net,"Raw_Data_UK_lines.xlsx")

df1

Load in GBreducednetwork = 58712 W
To replicate this system loading and generation, GBreducednetwork is multiplied by the scaling factor of 1
0
1
2


FileCreateError: [Errno 13] Permission denied: 'Raw_Data_UK_lines.xlsx'

In [42]:
pp.runpp(net)
pp.to_excel(net,"Raw_Data_UK.xlsx")
